In [11]:
import pandas as pd
import json
import requests

In [86]:
bearer_token = ""

hashtag = "Hamburg"
dataset_name = hashtag
num_tweets = 50
allow_retweets = False


In [82]:
# Step 1 : Make the request to the API
# We search for tweets containing the hashtag and their author



# Create a folder for the dataset if it doesn't exist
try:
    import os
    os.mkdir("data/" + dataset_name)
except:
    pass

# Store the response in a json file
def get_tweets(hashtag):

    # Create the url
    retweet_string = ""
    if not allow_retweets:
        retweet_string = "%20-is%3Aretweet"

    url = "https://api.twitter.com/2/tweets/search/recent?query=%23" + hashtag + retweet_string + "&max_results=" + str(num_tweets) + "&expansions=author_id&tweet.fields=id,created_at,text,author_id,in_reply_to_user_id,public_metrics,possibly_sensitive,lang&user.fields=id,created_at,name,username,protected,verified,profile_image_url,location,url,description,pinned_tweet_id,public_metrics"
    res = requests.get(url, headers={"Authorization": "Bearer " + bearer_token})
    
    # Check if the request was successful
    if (res.json().get("errors")):
        print(res.json().get("errors"))

    # Save the results to a json file
    with open("data/" + dataset_name + "/response.json", "w") as f:
        json.dump(res.json(), f)

get_tweets(hashtag)



In [87]:
# Step 2 : Extract the tweets from the response
def extract_tweets(dataset_name):

    # Load the json file
    with open("data/" + dataset_name + "/response.json", encoding="utf8") as f:
        data = json.load(f)

    # Convert the response to a pandas dataframe
    res_df = pd.DataFrame(data["data"])
    
    # Save the dataframe to a csv file
    res_df.to_csv("data/" + dataset_name + "/tweets.csv", index=False)

extract_tweets(dataset_name)

In [77]:
# Print some tweets in the dataset
tweets_file_name = "data/" + dataset_name + "/tweets.csv"

tweets_df = pd.read_csv(tweets_file_name)
tweets_df.head()

,author_id,lang,possibly_sensitive,text,created_at,id,public_metrics
0,1050404354887507969,es,False,RT @melu25: Yo hoy: Yo el resto...,2022-05-15T14:44:39.000Z,1525849649332080640,"{'retweet_count': 3342, 'reply_count': 0, 'lik..."
1,1507473751033757701,en,False,RT @senasleyla: Germany watching Spain and Uni...,2022-05-15T14:44:39.000Z,1525849649155870720,"{'retweet_count': 1282, 'reply_count': 0, 'lik..."
2,1239870359807156225,es,False,RT @Eurovision_ESP: De lo que estoy convencido...,2022-05-15T14:44:39.000Z,1525849648832946176,"{'retweet_count': 293, 'reply_count': 0, 'like..."
3,703133095,es,False,RT @Franlamadriddj: Lugar donde ir a partir de...,2022-05-15T14:44:39.000Z,1525849648686153729,"{'retweet_count': 2031, 'reply_count': 0, 'lik..."
4,720669223211679744,es,False,RT @generalitat: #BenidormFest2023 🏝️🎶\n\n🗣️ @...,2022-05-15T14:44:39.000Z,1525849648497303552,"{'retweet_count': 293, 'reply_count': 0, 'like..."


In [88]:
# Step 3 : Get the authors of the tweets

# Call the API to get the authors of the tweets
def get_authors(author_id_list):
    
    # Generate a string of author ids
    autor_string = ""
    for author_id in author_id_list:
        autor_string += str(author_id) + ","
    autor_string = autor_string[:-1] # remove last comma

    # request the listed authors to the API
    url = "https://api.twitter.com/2/users?ids=" + autor_string + "&user.fields=id,created_at,name,username,verified,profile_image_url,location,description,pinned_tweet_id"
    res = requests.get(url, headers={"Authorization": "Bearer " + bearer_token})

    # Check if there is an error
    if res.json().get("errors"):
        print(res.json()["errors"])

    # convert the response to a dataframe
    res_df = pd.DataFrame(res.json()["data"])

    # Save the dataframe to a csv file in the right folder
    res_df.to_csv("data/" + dataset_name + "/authors.csv", index=False)

def extract_authors(dataset_name):
    
    # Load the json file
    with open("data/" + dataset_name + "/response.json", encoding="utf8") as f:
        data = json.load(f)

    # Convert the response to a pandas dataframe
    res_df = pd.DataFrame(data["includes"]["users"])

    # Save the authors to a csv file
    res_df.to_csv("data/" + dataset_name + "/authors.csv", index=False)


extract_authors(dataset_name)


In [80]:
# Check if we know all the authors of the tweets
def check_authors(dataset_name):

    res = True

    # Load the tweets file
    tweets_df = pd.read_csv("data/" + dataset_name + "/tweets.csv")

    # Get the list of author ids
    author_id_list = tweets_df["author_id"].unique()

    # Check if the authors are in the authors file
    authors_df = pd.read_csv("data/" + dataset_name + "/authors.csv")

    for author_id in author_id_list:
        if author_id not in authors_df["id"].unique():
            print("Author id " + str(author_id) + " not found in the authors file")
            res = False

    if (res):
        print("All the tweets authors are in the authors file")


# Import the athors from the file
authors_df = pd.read_csv("data/" + dataset_name + "/authors.csv")

# Print some of the dataframe
check_authors(dataset_name)
authors_df[["id", "username", "name", "description"]].head()

All the tweets authors are in the authors file


,id,username,name,description
0,1050404354887507969,Marianavarrogu1,Magui 🔥,rise up\n\n🍺❤
1,1507473751033757701,FenronF1,FenronF1,Creemos en el plan de Fernando y de Ferrari.\n...
2,1239870359807156225,lucxjk,🍸,👉👈
3,703133095,Alejandra19_98,Ale 🌼,"historiadora, aunque eso no significa que no l..."
4,720669223211679744,francescoluigi_,Fran,NaN
